## IMPORTAR LAS LIBRERIAS

Actualizar a las que se usen finalmente en tu proyecto.

In [24]:
import numpy as np
import pandas as pd
import cloudpickle

#Automcompletar rápido
%config IPCompleter.greedy=True

from janitor import clean_names

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

## CARGAR LOS DATOS

### Ruta del proyecto

In [25]:
ruta_proyecto = 'C:/Users/isaac/Google Drive/DS4B/CursoMachineLearningPython/03_MACHINE_LEARNING/08_CASOS/00_PROYECTO1'

### Nombre del fichero de datos

In [26]:
nombre_fichero_datos = 'contratacion_fondos.csv'

### Cargar los datos

In [27]:
ruta_completa = ruta_proyecto + '/02_Datos/01_Originales/' + nombre_fichero_datos

df = pd.read_csv(ruta_completa,index_col=0)

### Seleccionar solo las variables finales

#### Cargar la lista de variables finales

In [28]:
nombre_variables_finales = ruta_proyecto + '/05_Resultados/' + 'variables_finales.pickle'

pd.read_pickle(nombre_variables_finales).sort_index().index.to_list()

['euribor3m_qt_mms',
 'euribor3m_qt_rs',
 'euribor3m_qt_ss',
 'euribor3m_yeo',
 'euribor3m_yeo_mms',
 'euribor3m_yeo_ss',
 'num_dias_ultimo_contacto_disc_ds',
 'num_dias_ultimo_contacto_disc_ds_mms',
 'num_dias_ultimo_contacto_disc_ds_rs',
 'num_dias_ultimo_contacto_disc_ds_ss',
 'resultado_campana_anterior_oe',
 'resultado_campana_anterior_oe_mms',
 'resultado_campana_anterior_oe_rs',
 'resultado_campana_anterior_oe_ss',
 'variacion_tasa_empleo_bin',
 'variacion_tasa_empleo_bin_mms',
 'variacion_tasa_empleo_bin_rs',
 'variacion_tasa_empleo_bin_ss']

#### Apuntar (manualmente) la lista de variables finales sin extensiones

In [29]:
variables_finales = ['euribor3m',
                   'num_dias_ultimo_contacto',
                   'resultado_campana_anterior',
                   'variacion_tasa_empleo'
                  ]

#### Crear la matriz de variables procesos (excel)

Ir a la plantilla de Excel "Fase Producción Plantilla Procesos" y crear la matriz de variables por procesos.

#### Actualizar las importaciones

Ir arriba a la celda de importacion de paquetes y actualizarlos con los que finalmente vamos a usar.

## ESTRUCTURA DE LOS DATASETS

### Corregir los nombres

In [30]:
df = clean_names(df)

### Eliminar registros

#### Por duplicados

In [31]:
df.drop_duplicates(inplace = True)

#### Por Nulos

In [32]:
df.dropna(thresh=3, inplace=True)

#### Para x

Quedarse solo con las de la lista.

In [33]:
x = df[variables_finales].copy()

#### Para y

Especificar la target.

In [34]:
target = 'contrata_fondos'

Crear el y.

In [35]:
y = df[target].copy()

## CREAR EL PIPELINE

### Instanciar calidad de datos

#### Crear la función

In [36]:
def calidad_datos(df):
    temp = df.astype({'euribor3m': 'float'})             
    
    temp.loc[df['resultado_campana_anterior'].isna(),'resultado_campana_anterior'] = 'nonexistent'
    
    temp['variacion_tasa_empleo'].clip(-20,10, inplace=True)
    
    return(temp)

#### Convertirla en transformer

In [37]:
hacer_calidad_datos = FunctionTransformer(calidad_datos)

### Instanciar transformación de variables

In [38]:
var_oe = ['resultado_campana_anterior']
oe = OrdinalEncoder(categories = [['nonexistent','failure','success']],
                    handle_unknown = 'use_encoded_value',
                    unknown_value = 10)

var_disc_ds = ['num_dias_ultimo_contacto']
disc_ds = KBinsDiscretizer(n_bins = 6, strategy = 'uniform', encode = 'ordinal')

var_bin = ['variacion_tasa_empleo']
bin = Binarizer(threshold=0)

var_qt = ['euribor3m']
qt = QuantileTransformer(output_distribution='normal')

var_ss = ['euribor3m','num_dias_ultimo_contacto','resultado_campana_anterior','variacion_tasa_empleo']
ss = StandardScaler()

### Crear el pipe del preprocesamiento

#### Crear el column transformer

In [39]:
ct = make_column_transformer(
    (oe, var_oe),
    (disc_ds, var_disc_ds),
    (bin, var_bin),
    (qt, var_qt),
    remainder='drop')

#### Crear el pipeline del preprocesamiento

In [40]:
pipe_prepro = make_pipeline(hacer_calidad_datos, 
                            ct)

### Instanciar el modelo

#### Instanciar el algoritmo

In [41]:
modelo = XGBClassifier(n_jobs = -1, 
                       verbosity = 0,
                       learning_rate = 0.01,
                       max_depth = 5,
                       reg_alpha = 0.1,
                       reg_lambda = 0.1,
                       n_estimators = 500,
                       use_label_encoder=False
                       )

#### Crear el pipe final de entrenamiento

In [42]:
pipe_entrenamiento = make_pipeline(pipe_prepro,modelo)

#### Guardar el pipe final de entrenamiento

In [43]:
nombre_pipe_entrenamiento = 'pipe_entrenamiento.pickle'

ruta_pipe_entrenamiento = ruta_proyecto + '/04_Modelos/' + nombre_pipe_entrenamiento

with open(ruta_pipe_entrenamiento, mode='wb') as file:
   cloudpickle.dump(pipe_entrenamiento, file)

#### Entrenar el pipe final de ejecución

In [44]:
pipe_ejecucion = pipe_entrenamiento.fit(x,y)

## GUARDAR EL PIPE

### Nombre del pipe final de ejecución

In [45]:
nombre_pipe_ejecucion = 'pipe_ejecucion.pickle'

### Guardar el pipe final de ejecución

In [46]:
ruta_pipe_ejecucion = ruta_proyecto + '/04_Modelos/' + nombre_pipe_ejecucion

with open(ruta_pipe_ejecucion, mode='wb') as file:
   cloudpickle.dump(pipe_ejecucion, file)